#### Import Libaries

In [1]:
import os
import random
import shutil
import csv
import pandas as pd
import xml.etree.ElementTree as ET
import cv2

In [6]:
os.getcwd()
%cd ./TubineDefectDector/data

[WinError 3] Sistem belirtilen yolu bulamıyor: './TubineDefectDector/data'
f:\İndirilenler\TurbineDefectDector-master_2\TurbineDefectDector-master


#### copy files using shutil

In [7]:
def copy_image_file(source,target):
    assert not os.path.isabs(source)
    try:
        shutil.copy(source, target)
    except IOError as e:
        print("Unable to copy file. %s" % e)
    except:
        print("Unexpected error:", sys.exc_info())

#### randomly split data into train and test folders

In [8]:
def split_images_files(filepath):
files = os.listdir(filepath)
for f in files:
    if round(random.random(),1) < 0.8:
        tgt = os.path.join('./train_files',f)
        scr = os.path.join(imgpath,f)
        copy_image_file(scr,tgt)
    else:
        tgt = os.path.join('./test_files',f)
        scr = os.path.join(imgpath,f)
        copy_image_file(scr,tgt)
# set path and list filenames

img_path = './images'
split_images_files(img_path)

IndentationError: expected an indented block (<ipython-input-8-e068e2a8d9af>, line 2)

#### Function to convert LabelImg XML output to a csv file. so that we can use in fRCNN model annotation input. 

In [7]:
def gereate_annotation_records():
    # open csv file and create write object
    annotated_file = open("annotated_data.csv", 'w')
    csvwriter = csv.writer(annotated_file)
    csvwriter.writerow(['filename', 'fault_type', 'xmin', 'xmax', 'ymin', 'ymax'])

    # loop over the xml files
    xml_path = './annotations'
    xml_files = os.listdir(xml_path)
    for f in xml_files:
        if f.endswith('xml'):
            # create tree and root of xml file
            tree = ET.parse(os.path.join(xml_path, f))
            root = tree.getroot()

            # loop over the elements in the xml file
            for element in root.findall('object'):
                # get the input fields
                name = element.find('name').text
                bndbox = element.find('bndbox')
                xmin = bndbox.find('xmin').text
                xmax = bndbox.find('xmax').text
                ymin = bndbox.find('ymin').text
                ymax = bndbox.find('ymax').text

                # create list with the values
                row = []
                new_path = f[:-4] + ".JPG" # skip last four elements of f -> .xml extension
                row.extend([new_path, name, xmin, xmax, ymin, ymax])

                # write them away to csv
                csvwriter.writerow(row)

    # close csv file after appending the data
    annotated_file.close()
    # open the file
    annotated_data = pd.read_csv('annotated_data.csv')

#### function to normalize defect types

In [9]:
def change_faulttype(ftype):
    switcher={
        'VG_PANEL':'VG_PANEL',
        'VG PANEL':'VG_PANEL',
        'VG _PANEL':'VG_PANEL',
        'VG PANEL MISSING':'VG_MISSING_TEETH',
        'VG_MISSING_TEETH':'VG_MISSING_TEETH',
        'MISSING PANELS':'VG_MISSING_TEETH',
        'VG _MISSING _TEETH':'VG_MISSING_TEETH',
        'PANELS MISSING':'VG_MISSING_TEETH',
        'LIGHTNING DENT':'SURFACE_DAMAGE_TY1',
        'LIGHTNING _RECEPTOR':'SURFACE_DAMAGE_TY1',
        'LIGHTNING_RECEPTOR':'SURFACE_DAMAGE_TY1',
        'DENT':'SURFACE_DAMAGE_TY1',
        'HIT BY SOME PARTICLES':'SURFACE_DAMAGE_TY1',
        'LIGHTNING DAMAGE':'SURFACE_DAMAGE_TY1',
        'SURFACE_DAMAGE_TY1':'SURFACE_DAMAGE_TY1'
     }
    return switcher.get(ftype,ftype)
 

In [10]:
annotated_data = pd.read_csv('annotated_data.csv')
annotated_data['fault_type'] = annotated_data.fault_type.apply(lambda x: x.upper() )
annotated_data['fault_type'] = annotated_data.fault_type.apply(lambda x: change_faulttype(x))

In [10]:
annotated_data.fault_type.value_counts()

NameError: name 'annotated_data' is not defined

#### generated annotatipn file for faster-RCNN model input

In [2]:
train_data = pd.DataFrame(index=[-1],columns=['input_format'])
train_file_path = './train_files'
train_files = os.listdir(train_file_path)
for f in train_files:
    if str(f).find('JPG') > 0 :
        image_fullpath= os.path.join(train_file_path,f)
        if len(annotated_data[annotated_data.filename ==f]) > 0:
            for _,row in annotated_data[annotated_data.filename ==f].iterrows():
                if (row.fault_type in ['VG_MISSING_TEETH','VG_PANEL','SURFACE_DAMAGE_TY1']):
              # add xmin, ymin, xmax, ymax and class as per the format required
                    xmin = row.xmin
                    xmax = row.xmax
                    ymin = row.ymin
                    ymax = row.ymax
                    faulttype = row.fault_type
                    data_str = image_fullpath + ',' + str(xmin) + ',' + str(ymin) + ',' + str(xmax) + ',' + str(ymax) + ',' + str(faulttype)
                    train_data.loc[train_data.index.max()+1] = [data_str]
        #else:
            #print(f)
            #data_str = image_fullpath + ',,,,,'
            #train_data.loc[train_data.index.max()+1] = [data_str]
train_data.drop([-1],inplace=True)
train_data.to_csv('train_annotate1.txt', header=None, index=None, sep=' ')

FileNotFoundError: [Errno 2] No such file or directory: './train_files'

In [ ]:
train_data =pd.read_csv('train_annotate10.txt', header=None, sep=',')
train_data.columns =['imagename','x1','y1','x2','y2','defecttype']
train_data[train_data.imagename.str.contains('0400.JPG')]

#### making annotated image

In [ ]:
def make_annotated_image(image_path,imgname):
    img=cv2.imread(imgname)
    for _,row in train_data[train_data.imagename.str.contains(imgname)].iterrows():
        print(row.defecttype)
        real_x1, real_y1,real_x2, real_y2 = (int(row.x1),int(row.y1),int(row.x2),int(row.y2))
        cv2.rectangle(img,(real_x1, real_y1), (real_x2, real_y2), (36,255,12),1)
        textLabel = row.defecttype
        (retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_COMPLEX,1,1)
        textOrg = (real_x1, real_y1-0)
        cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (0, 0, 0), 2)
        cv2.rectangle(img, (textOrg[0] - 5,textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (255, 255, 255), -1)
        cv2.putText(img, textLabel, textOrg, cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 1)

    cv2.imwrite(image_name1+'_annot3.jpg',img)

In [ ]:
image_name1 = "DJI_0212"
image_name ='./train_files/DJI_0212.JPG'
make_annotated_image(image_name,image_name1):